In [5]:
import os
import pandas as pd
from collections import Counter
from tqdm import tqdm
import spacy
import en_core_web_sm
import json
pos_tag=[]
nlp = en_core_web_sm.load()

# Creating sentence-[prdiction list] ouput for testing purposes

In [66]:
file_list=os.listdir("txt_2/txt")
x_list=[]
for file in file_list:
    f=open("txt_2/txt/"+file, "r")
    x_list.append(f.read())
    
test=pd.DataFrame(x_list)

In [11]:
# Check for length if needed
test['total_length'] = test[0].apply(len)
test.drop("total_length",axis=1,inplace=True)

In [25]:
# Adding output to sentences from LSTM-CRF
with open("targer-master/out.json") as json_file:
    data=json.load(json_file)

In [27]:
formatted_output=pd.DataFrame()
formatted_output["sentences"]=test[0]
formatted_output["tags"]=data

In [109]:
np.savetxt(r'formatted_output.txt', formatted_output.values, fmt='%s', delimiter=' ')

# Continuing from last - making CoNLL-2003 formatted data from simple sentences

In [12]:
sentence_num=[]
character=[]
for i,row in test.iterrows():
    character.append("null")
    sentence_num.append(i)

    for j in str(row[0]).split():
        sentence_num.append(i)
        character.append(j)

In [13]:
test_final=pd.DataFrame()

In [14]:
test_final["character"]=character
test_final["sentence_num"]=sentence_num

In [15]:
test_final["character"].replace("null",np.nan,inplace=True)

In [16]:
test_final["character"]=test_final["character"].str.lower()

In [55]:
#test_final.to_csv("test.txt",index=False)

In [17]:
pos_tag=[]
nlp = en_core_web_sm.load()

In [18]:
test_final.fillna("-",inplace=True)

In [19]:
words = list(set(test_final["character"].values))
n_words = len(words); n_words

15248

In [20]:
unique_words=test_final['character'].unique()

In [22]:
# Adding POS to CoNLL formatted data
pos_tag={}
for ind in tqdm(range(len(unique_words))):
    doc=nlp(unique_words[ind])
    for token in doc:
        pos_tag[unique_words[ind]]=token.tag_
        
        
train_words=list(pos_tag.keys())
train_pos=list(pos_tag.values())

pos_df=pd.DataFrame()
pos_df['character']=train_words
pos_df['pos']=train_pos

train_df_pos=test_final.merge(pos_df,on='character',how='left')
# Converting text to lower-case
train_df_pos["character"]=train_df_pos["character"].str.lower()

In [27]:
#for index, row in train_df_pos.iterrows():
#    if row["character"]=="-":
#        row["character"][index]=np.nan
#        row["pos"][index]=np.nan

In [28]:
train_df_pos["character"].replace("-",np.nan,inplace=True)
train_df_pos["pos"].replace(":",np.nan,inplace=True)
train_df_pos.fillna("",inplace=True)
train_df_pos["tag1"]="O"
train_df_pos["tag2"]="O"
train_df_pos.loc[(train_df_pos['pos'] == ""), 'tag1'] = ""
train_df_pos.loc[(train_df_pos['pos'] == ""), 'tag2'] = ""

In [29]:
train_df_pos.drop("sentence_num",axis=1,inplace=True)

In [34]:
#test_final.to_csv("test.txt",index=False)
np.savetxt(r'test_big.txt', train_df_pos.values, fmt='%s', delimiter=' ')

# Random - Take only important

In [6]:
import json
with open("targer-master/out.json") as json_file:
    data=json.load(json_file)

In [8]:
file_list=os.listdir("txt_2/txt")
x_list=[]
for file in file_list:
    f=open("txt_2/txt/"+file, "r")
    x_list.append(f.read())
    
test=pd.DataFrame(x_list)

sentence_num_2=[]
character_2=[]
for i,row in test.iterrows():
    #character.append("null")
    #sentence_num_2.append(i)

    for j in str(row[0]).split():
        sentence_num_2.append(i)
        character_2.append(j)

In [9]:
flattened_list = [y for x in data for y in x]

In [10]:
op=pd.DataFrame()

In [11]:
op["character"]=character_2

In [12]:
op["predicted_label"]=flattened_list

In [13]:
value=op[op["predicted_label"]=="B-PER"]["character"].value_counts()
value=dict(value)

In [14]:
#large_occuring=[]
#for item in value:
#    if value[item]>50:
#        large_occuring.append(item)

In [29]:
all_occuring=[]
for item in value:
    all_occuring.append(item)

In [30]:
include=['MARTIN','DICK','ELIJAH''HARRISON','IDEALISM','CAMERON','MANUEL''GREGSON','JONES',"O'BRIEN",'PAT','DAYLIGHT','NIMROD','PIERRE','LORD',"JEANNE'S",'BILL','DAUGHTRY',"SAXON'S",'ELI','PASCAL','STEWART','ERNEST''OOLONG',
 'OUTWARDLY',"DENNIN'S",'SANDEL','GOD','MACDOUGALL','FRANCOIS','SAXON','FERGUSON','JESSE','ISAAC','STEVE','CURLY','HARRY','GEORGE','ARTHUR','PHILIP','WHITE'"DOANE'S",
 'JOHNNY','BOB','PHIL','BURKE','JACQUES','SARAH','JERRY','GABRIEL','WHITTEMORE','THORPE','JOE','TOM','CHAUNCEY','CHEROKEE''SHORTY']

In [31]:
exclude=list(set(all_occuring).difference(set(include)))

In [15]:
exclude=["YOU'LL","YOU'RE","YOU'VE","WE'LL",'SIR',"SHOULDN'T","SHE'D",'ORGANIZED','MR','MRS','MOTHER','MATHEMATICAL','MADAM',"MA'AM",
'LADY','IGNORANTLY',"I'LL","I'M","I'VE",'HYPOTHESES',"HE'S","HE'LL",'HELLO',"HADN'T",'FOURTEEN','ELDORADO','DEJECTED','COOK',
"CHILDREN'S","BOY'S",'ANYWAYS']

In [16]:
for index,row in op.iterrows():
    if row["character"] in exclude:
        op["predicted_label"].iloc[index]="O"

In [17]:
#op["predicted_label"].value_counts()

In [18]:
#op[op["predicted_label"]=="B-ORG"]

In [19]:
op["sentence_num"]=sentence_num_2

In [111]:
#op["predicted_label"].value_counts()

In [112]:
#op[op["predicted_label"]=="B-PER"]

In [113]:
#op.to_csv("Final_Output.csv",index=False)

In [20]:
lol=False
index1=0
hemant=[]
for index,row in op.iterrows():
    index1+=1
#for ind in op.index:
    if row["predicted_label"]=="B-PER":
        #line = pd.DataFrame({"character": "|", "predicted_label": "O","sentence_num":-999}, index=[index-1])
        #op = pd.concat([op.iloc[:index], line, op.iloc[index:]]).reset_index(drop=True)
        #i+=2
        op = pd.DataFrame(np.insert(op.values, index1-1, values=["|","O",row["sentence_num"]], axis=0),columns = op.columns)
        op = pd.DataFrame(np.insert(op.values, index1+1, values=[">","O",row["sentence_num"]], axis=0),columns = op.columns)
        index1+=2
        hemant.append(row["sentence_num"])
        
        #print(index)

In [21]:
#op[op["predicted_label"]=="B-PER"]

In [22]:
#op[op["sentence_num"]==8]

In [23]:
#op.iloc[450:470]

In [24]:
# Get sentences from CoNLL-2003 formatted data

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [w for w in s['character'].values.tolist()]
        self.grouped = self.data.groupby('sentence_num').apply(agg_func)
        self.sentences = self.grouped
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(op)
sentences = getter.sentences

In [25]:
sentences_final=[]
for i in range(len(sentences)):
    sentences_final.append(' '.join(word for word in sentences[i]))

In [26]:
final=pd.DataFrame()
final["sentences"]=sentences_final

In [27]:
sentences_final_2=list(final["sentences"])

In [29]:
file_list=os.listdir("txt_2/txt/")

In [31]:
hemant1=list(set(hemant))

In [32]:
file_name_hemant=[]
for i in range(len(file_list)):
    if i in hemant1:
        file_name_hemant.append(file_list[i])
    f = open("Updated_txt_big_95_2/"+file_list[i],'w')
    f.write(sentences_final_2[i])
    #np.savetxt(r"Updated_txt/"+file_list[i], sentences_final_2[i])

In [36]:
with open('filenames.txt', 'w') as f:
    for item in file_name_hemant:
        f.write("%s\n" % item)

# Edit CoNLL-2003 dataset using the following code

In [3]:
#lol = pd.read_fwf("C:/Users/Sreyan/Desktop/corpus_mixed/train.txt")

In [77]:
# Reading the data
file=open("C:/Users/Sreyan/Desktop/NER/targer-master/data/NER/CoNNL_2003_shared_task/train.txt","r")
lines=file.readlines()
new_data=pd.DataFrame(lines)

In [80]:
columns=new_data[0].str.split(" ",n=3,expand=True)

In [81]:
columns=columns.iloc[1:,:]

In [83]:
columns[0].replace("\n"," ",inplace=True)
columns[1] = [str(i or ' ') for i in columns[1]]
columns[2] = [str(i or ' ') for i in columns[2]]
columns[3] = [str(i or ' ') for i in columns[3]]
#columns[1] = [str(i or ' ') for i in columns[1]]
#columns[1].replace(None," ",inplace=True)
#columns[2].replace(None," ",inplace=True)
#columns[3].replace(None," ",inplace=True)

In [84]:
columns[3] = columns[3].map(lambda x: x.lstrip('').rstrip('\n'))

In [85]:
columns[0]=columns[0].str.lower()

In [94]:
# Add and drop columns if required
columns.drop([1,2],axis=1,inplace=True)

In [95]:
# Extra code for removing MISC NER Tags
columns[3].replace("B-MISC","O",inplace=True)
columns[3].replace("I-MISC","O",inplace=True)

In [98]:
# Rename the columns according to requirement
columns.rename(columns = {0: "character", 1:"ner"}, inplace = True)

In [99]:
# Save the dataset after editing
np.savetxt(r"C:/Users/Sreyan/Desktop/conll3.txt", columns.values, fmt='%s', delimiter=' ')

#  Creating WNUT NER Data fit for adding to CoNLL-2003


In [153]:
# Reading the text file
file_wnut=open("C:/Users/Sreyan/Desktop/NER/NER-datasets-master/WNUT17/wnut_dev.txt","r",encoding="utf8")
lines_wnut=file_wnut.readlines()
new_data_wnut=pd.DataFrame(lines_wnut)

In [156]:
new_data_wnut.head()

In [157]:
columns_wnut=new_data_wnut[0].str.split("\t",n=2,expand=True)

In [158]:
# Making it fit for editing
columns_wnut[0].replace("\n"," ",inplace=True)
columns_wnut[1] = [str(i or ' ') for i in columns_wnut[1]]
#columns_wnut[2] = [str(i or ' ') for i in columns_wnut[2]]
#columns_wnut[3] = [str(i or ' ') for i in columns_wnut[3]]

In [159]:
columns_wnut[1] = columns_wnut[1].map(lambda x: x.lstrip('').rstrip('\n'))

In [160]:
columns_wnut[0]=columns_wnut[0].str.lower()

In [163]:
# Editing NER Tags to make it similar to CoNLL-2003
columns_wnut[1].replace("B-person","B-PER",inplace=True)
columns_wnut[1].replace("I-person","I-PER",inplace=True)
columns_wnut[1].replace("I-location","I-LOC",inplace=True)
columns_wnut[1].replace("B-location","B-LOC",inplace=True)
columns_wnut[1].replace("B-corporation","B-ORG",inplace=True)
columns_wnut[1].replace("I-corporation","I-ORG",inplace=True)
columns_wnut[1].replace("I-product","O",inplace=True)
columns_wnut[1].replace("B-product","O",inplace=True)
columns_wnut[1].replace("I-group","O",inplace=True)
columns_wnut[1].replace("B-group","O",inplace=True)
columns_wnut[1].replace("B-creative-work","O",inplace=True)
columns_wnut[1].replace("I-creative-work","O",inplace=True)
columns_wnut[1].replace(""," ",inplace=True)
columns_wnut[1].replace("-","O",inplace=True)

In [164]:
columns_wnut.replace(" ","-",inplace=True)

In [165]:
columns_wnut["character"]=columns_wnut[0]

In [166]:
words = list(set(columns_wnut[0].values))
n_words = len(words); n_words

3539

In [167]:
unique_words=columns_wnut[0].unique()

In [168]:
# Creating POS for words
pos_tag={}
for ind in tqdm(range(len(unique_words))):
    doc=nlp(unique_words[ind])
    for token in doc:
        pos_tag[unique_words[ind]]=token.tag_

100%|█████████████████████████████████████████████████████████████████████████████| 3539/3539 [00:20<00:00, 174.52it/s]


In [169]:
train_words=list(pos_tag.keys())
train_pos=list(pos_tag.values())

In [170]:
pos_df=pd.DataFrame()
pos_df['character']=train_words
pos_df['pos']=train_pos

In [171]:
# Merging with original words
wnut=columns_wnut.merge(pos_df,on='character',how='left')

In [172]:
wnut["tag"]=wnut[1]
wnut.drop([0,1],axis=1,inplace=True)

In [173]:
for index,row in wnut.iterrows():
    if row["pos"]==":":
        wnut["character"].iloc[index]=" "
        wnut["tag"].iloc[index]=" "
        wnut["pos"].iloc[index]=" "

In [174]:
wnut[wnut["tag"]==""]["character"].value_counts()

Series([], Name: character, dtype: int64)

In [202]:
np.savetxt(r'wnut.txt', wnut.values, fmt='%s', delimiter=' ')

In [120]:
conatenated_dev=pd.concat([columns_dev,wnut],axis=0,ignore_index=True)

In [126]:
np.savetxt(r'conatenated_dev.txt', conatenated_dev.values, fmt='%s', delimiter=' ',encoding="utf8")

# Editing sentences with extra spaces between actual word and 's


In [29]:
if "'s" in split_sentence:
    if "]" in split_sentence[split_sentence.index("'s")-1]:
        split_sentence[split_sentence.index("'s")-1]=split_sentence[split_sentence.index("'s")-1][:-1]+"'s"+"]"
    else:
        split_sentence[split_sentence.index("'s")-1]=split_sentence[split_sentence.index("'s")-1]+"'s"
    split_sentence.remove("'s")